# Dependencias

In [ ]:
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com

In [ ]:
!pip install rasterio

In [ ]:
# Librerias
import os
import json
import glob
import numpy as np
import rasterio
from rasterio.features import rasterize
from shapely.geometry import Polygon
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import (
    adjusted_rand_score,
    adjusted_mutual_info_score,
    classification_report,
    f1_score,
    accuracy_score,
    jaccard_score
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.manifold import TSNE
from cuml.naive_bayes import GaussianNB
from cuml.svm import SVC
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.metrics import accuracy_score as cu_accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from scipy.ndimage import uniform_filter
from tqdm import tqdm
import matplotlib.pyplot as plt
import joblib
from google.colab import files


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Features

In [ ]:
# ==========================================
# 📁 RUTAS
# ==========================================
base_dir = "/content/drive/MyDrive/Imagenes"
mask_dir = os.path.join(base_dir, "Recortes_Mask")
rgb_dir  = os.path.join(base_dir, "Recortes_RGB")

print("📂 Carpetas detectadas:")
print("MASKs:", len(glob.glob(os.path.join(mask_dir, "*.tiff"))))
print("RGBs :", len(glob.glob(os.path.join(rgb_dir, "*.tiff"))))

# ==========================================
# 🧠 FUNCIÓN: extraer características (RGB + índices)
# ==========================================

def extraer_features(img):
    img = img.astype(np.float32)

    R, G, B = img[..., 0], img[..., 1], img[..., 2]

    # --- Calcular medias locales (como ya hacías) ---
    R_mean = uniform_filter(R, size=3)
    G_mean = uniform_filter(G, size=3)
    B_mean = uniform_filter(B, size=3)

    # --- Combinar características ---
    features = np.stack([
        R, G, B,
        R_mean, G_mean, B_mean
    ], axis=-1)

    return features

# ==========================================
# 📚 CARGA Y PREPARACIÓN DE DATOS
# ==========================================
X_data, y_data = [], []

mask_files = sorted(glob.glob(os.path.join(mask_dir, "*.tiff")))

for mask_path in tqdm(mask_files):
    name = os.path.splitext(os.path.basename(mask_path))[0]
    rgb_path = os.path.join(rgb_dir, name + ".tiff")

    if not os.path.exists(rgb_path):
        print(f"⚠️ No se encontró RGB para {name}")
        continue

    # --- Cargar imagen RGB ---
    with rasterio.open(rgb_path) as src:
        img = src.read().transpose(1, 2, 0)
        img_shape = img.shape[:2]

    # --- Cargar máscara ---
    with rasterio.open(mask_path) as src:
        mask = src.read(1)

    # --- Extraer características ---
    feats = extraer_features(img)
    X = feats.reshape(-1, feats.shape[2])
    y = (mask > 0).astype(np.uint8).flatten()

    # --- Muestreo balanceado ---
    pos_idx = np.where(y == 1)[0]
    neg_idx = np.where(y == 0)[0]
    if len(pos_idx) == 0 or len(neg_idx) == 0:
        continue
    n_samples = min(len(pos_idx), len(neg_idx), 10000)
    sample_idx = np.concatenate([
        np.random.choice(pos_idx, n_samples, replace=False),
        np.random.choice(neg_idx, n_samples, replace=False)
    ])

    X_data.append(X[sample_idx])
    y_data.append(y[sample_idx])

print(f"📊 Imágenes procesadas: {len(X_data)}")

if len(X_data) == 0:
    raise ValueError("❌ No se generó ningún conjunto de entrenamiento. Revisa los nombres o las rutas.")

# --- Concatenar todo ---
X_data = np.concatenate(X_data)
y_data = np.concatenate(y_data)
print("✅ Dataset final:", X_data.shape, y_data.shape)

# ML: Tradicional

In [ ]:
# RandomForest
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
rf = cuRF(n_estimators=20, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))
print("🎯 IoU:", jaccard_score(y_test, y_pred))

In [ ]:
# Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)

print(classification_report(y_test, y_pred_gnb))
print("🎯 IoU:", jaccard_score(y_test, y_pred_gnb))

In [ ]:
# Desicion Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print(classification_report(y_test, y_pred_dt))
print("🎯 IoU:", jaccard_score(y_test, y_pred_dt))

# ML: Deep Learning

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

# Normalizamos los valores (muy importante para redes neuronales)
X_train = X_train / 255.0
X_test = X_test / 255.0

# ==========================================
# 🧠 MODELO DE RED NEURONAL DENSA
# ==========================================
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # salida binaria
])

# ==========================================
# ⚙️ COMPILAR EL MODELO
# ==========================================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ==========================================
# 🚀 ENTRENAMIENTO
# ==========================================
early_stop = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=256,
    callbacks=[early_stop],
    verbose=1
)

# ==========================================
# 🧾 EVALUACIÓN
# ==========================================
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print("🎯 IoU:", jaccard_score(y_test, y_pred))


# No supervisado

In [ ]:
# En feature tomar solo de 1 a 10 imagenes para que el rendimiento sea optimo
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.20, random_state=42)

n_features = X_train.shape[1]
n_components = max(1, n_features // 2)

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca  = pca.transform(X_test)

X_train_concat = np.concatenate([X_train, X_train_pca], axis=1)
X_test_concat  = np.concatenate([X_test, X_test_pca], axis=1)


def build_dnn(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=SGD(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

def train_and_eval(Xtr, Xte, title):
    print("\n======================================")
    print("MODELO:", title)
    print("======================================")

    model = build_dnn(Xtr.shape[1])

    model.fit(Xtr, y_train,validation_data=(Xte, y_test),epochs=3,batch_size=256,verbose=0)

    # Train predictions
    y_pred_train = (model.predict(Xtr) > 0.5).astype(int)
    acc_train = accuracy_score(y_train, y_pred_train)
    f1_train  = f1_score(y_train, y_pred_train)

    # Test predictions
    y_pred_test = (model.predict(Xte) > 0.5).astype(int)
    acc_test = accuracy_score(y_test, y_pred_test)
    f1_test  = f1_score(y_test, y_pred_test)

    print(f"Train Accuracy: {acc_train:.4f} | Train F1: {f1_train:.4f}")
    print(f"Test  Accuracy: {acc_test:.4f} | Test  F1: {f1_test:.4f}")

    return (acc_train, f1_train, acc_test, f1_test)

# --- 1 Solo Features originales
res_orig = train_and_eval(X_train, X_test, "Solo FEAT")

# --- 2️ Features + PCA
res_concat = train_and_eval(X_train_concat, X_test_concat, "FEAT + PCA")

# --- 3️ Solo PCA
res_pca = train_and_eval(X_train_pca, X_test_pca, "Solo PCA")

In [ ]:
# 1Árbol de decisión en un subconjunto
X_sub, _, y_sub, _ = train_test_split(X_data, y_data, test_size=0.8, random_state=42)

dt = DecisionTreeClassifier(max_depth=4, random_state=0)
dt.fit(X_sub, y_sub)

importances = dt.feature_importances_
top2 = np.argsort(importances)[::-1][:2]

print("\nTop 2 características más importantes del árbol:")
print("Indices:", top2.tolist())

# PCA con 2 componentes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_data)


# TSNE con 2 componentes
tsne = TSNE(n_components=2, random_state=0, perplexity=30)
X_tsne = tsne.fit_transform(X_data)

# 4. Normalizar + KMeans y DBSCAN
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_data)

# KMeans
kmeans = KMeans(n_clusters=len(np.unique(y_data)), random_state=0)
kmeans_labels = kmeans.fit_predict(X_norm)

# DBSCAN
dbscan = DBSCAN(eps=0.7, min_samples=20, n_jobs=-1)
db_labels = dbscan.fit_predict(X_norm)

# 5. Función de scatter
def scatter(ax, data, labels, title):
    ax.scatter(data[:,0], data[:,1], s=8, c=labels, cmap="tab10")
    ax.set_title(title)

# 6. FIGURA 3×3 (CON TSNE, SIN SUBMUESTREO)
fig, axes = plt.subplots(3, 3, figsize=(14, 14))

# ---- Fila 1  PCA ----
scatter(axes[0,0], X_pca, kmeans_labels, "PCA - KMeans")
scatter(axes[0,1], X_pca, db_labels, "PCA - DBSCAN")
scatter(axes[0,2], X_pca, y_data, "PCA - Clases Dataset")

# ---- Fila 2  TSNE ----
scatter(axes[1,0], X_tsne, kmeans_labels, "TSNE - KMeans")
scatter(axes[1,1], X_tsne, db_labels, "TSNE - DBSCAN")
scatter(axes[1,2], X_tsne, y_data, "TSNE - Clases Dataset")

# ---- Fila 3  Mejores 2 features ----
X_best2 = X_data[:, top2]
scatter(axes[2,0], X_best2, kmeans_labels, "Best2 - KMeans")
scatter(axes[2,1], X_best2, db_labels, "Best2 - DBSCAN")
scatter(axes[2,2], X_best2, y_data, "Best2 - Clases Dataset")

plt.tight_layout()
plt.show()

# Pruebas

In [ ]:
# ==========================================
# 📸 CARGAR IMAGEN Y MÁSCARA
# ==========================================

test_img_path = sorted(glob.glob(os.path.join(rgb_dir, "*.tiff")))[13]
test_mask_path = sorted(glob.glob(os.path.join(mask_dir, "*.tiff")))[13]

with rasterio.open(test_img_path) as src:
    img_test = src.read().transpose(1, 2, 0)

with rasterio.open(test_mask_path) as src:
    mask_test = src.read(1)

# ==========================================
# 🧩 EXTRAER FEATURES (debe coincidir con entrenamiento)
# ==========================================
img_feats = extraer_features(img_test).reshape(-1, 6)
img_feats = img_feats / 255.0  # normalizar igual que en entrenamiento

# ==========================================
# 🧠 PREDICCIÓN CON LA RED NEURONAL
# ==========================================
mask_pred_prob = model.predict(img_feats, batch_size=4096, verbose=1)
mask_pred = (mask_pred_prob > 0.5).astype(np.uint8).reshape(img_test.shape[:2])

# ==========================================
# 🎨 VISUALIZACIÓN
# ==========================================
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Imagen original")
plt.imshow(img_test)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Máscara predicha")
plt.imshow(mask_pred, cmap='gray')
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Máscara real (Ground Truth)")
plt.imshow(mask_test, cmap='gray')
plt.axis("off")

plt.tight_layout()
plt.show()